In [10]:
import wandb
import pandas as pd
import os 
import  tempfile
# set proxy
#os.environ['http_proxy'] = 'http://genproxy.corp.amdocs.com:8080'
#os.environ['https_proxy'] = 'http://genproxy.corp.amdocs.com:8080'
run = wandb.init(project="nyc_airbnb", group="eda", save_code=True)
local_path = wandb.use_artifact("sample.csv:latest").file()
#local_path="/tmp/sample1.csv"
df = pd.read_csv(local_path)

In [8]:


#import pandas_profiling
import ydata_profiling as pp

profile = pp.ProfileReport(df)
profile.to_widgets()


Summarize dataset:   0%|          | 0/5 [00:00<?, ?it/s]

/home/meirof/miniconda3.10/envs/mlflow-36940a21f0367bbab9d9e81b1b836b0babaae4af/lib/python3.10/site-packages/ydata_profiling/model/typeset.py:208: FutureWarning: is_categorical_dtype is deprecated and will be removed in a future version. Use isinstance(dtype, CategoricalDtype) instead
  is_valid_dtype = pdt.is_categorical_dtype(series) and not pdt.is_bool_dtype(
/home/meirof/miniconda3.10/envs/mlflow-36940a21f0367bbab9d9e81b1b836b0babaae4af/lib/python3.10/site-packages/ydata_profiling/model/typeset.py:125: FutureWarning: is_categorical_dtype is deprecated and will be removed in a future version. Use isinstance(dtype, CategoricalDtype) instead
  not pdt.is_categorical_dtype(series)
/home/meirof/miniconda3.10/envs/mlflow-36940a21f0367bbab9d9e81b1b836b0babaae4af/lib/python3.10/site-packages/ydata_profiling/model/typeset.py:208: FutureWarning: is_categorical_dtype is deprecated and will be removed in a future version. Use isinstance(dtype, CategoricalDtype) instead
  is_valid_dtype = pdt.i

Generate report structure:   0%|          | 0/1 [00:00<?, ?it/s]

Render widgets:   0%|          | 0/1 [00:00<?, ?it/s]

In [11]:
# Drop outliers
min_price = 10
max_price = 350
idx = df['price'].between(min_price, max_price)
df = df[idx].copy()
# Convert last_review to datetime
df['last_review'] = pd.to_datetime(df['last_review'])

In [12]:
with tempfile.TemporaryDirectory() as tmp_dir:
    artifact_name = f"post_eda_data.csv"
    temp_path = os.path.join(tmp_dir, artifact_name)
    df.to_csv(temp_path,index = False)
    artifact = wandb.Artifact(
                    name=artifact_name,
                    type="raw-data",
                    description=f" Data after outliers",
            )
    artifact.add_file(temp_path)
    run.log_artifact(artifact)
    run.log_artifact(artifact)
            # This waits for the artifact to be uploaded to W&B. If you
            # do not add this, the temp directory might be removed before
            # W&B had a chance to upload the datasets, and the upload
            # might fail
    artifact.wait()
run.finish()

wandb: WARNING Source type is set to 'repo' but some required information is missing from the environment. A job will not be created from this run. See https://docs.wandb.ai/guides/launch/create-job
